# Image generator

This notebook allows to generate synthetic datasets of face images sampled from StyleGAN2. It saves both the images and their latent codes (in $\mathcal{Z}$).

# Part 1 Retrieving the model
If you have already retrieved the model, just %cd to the folder containing stylegan2-ada-pytorch/ and go to part 2.
(Do not cd into stylegan2-ada-pytorch/, just to the folder containing it.)

Install StyleGAN2-ada-pytorch dependencies

In [1]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

DEPRECATION: Loading egg at c:\users\nmadali\appdata\roaming\python\python311\site-packages\channelnorm_cuda-0.0.0-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nmadali\appdata\roaming\python\python311\site-packages\correlation_cuda-0.0.0-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nmadali\appdata\roaming\python\python311\site-packages\resample2d_cuda-0.0.0-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new r

# Part 2 Generating and saving images

In [2]:
import torch, pickle
import functools

from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

import numpy as np
from tqdm import tqdm
import os

In [3]:
!ls -l

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [4]:

!git clone https://github.com/rkuo2000/stylegan2-ada-pytorch
%cd stylegan2-ada-pytorch

In [ ]:
if not os.path.isdir("pretrained"):
  !mkdir pretrained
  %cd pretrained
  !wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
  %cd ../

In [5]:
# Retrieving Generator
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

with open('pretrained/ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()
if device_name == "cpu":
  G.synthesis.forward = functools.partial(G.synthesis.forward, force_fp32=True)


%cd ..

C:\Users\nmadali\Nabil\GANSpace-Reimplementation-main


CD into the desired location for your dataset:

In [6]:
# Run if those directories do not already exist
!mkdir test_UnlabeledImages
!mkdir test_ImageLatentCodes

In [7]:
if not os.path.exists('test_UnlabeledImages'):
    os.makedirs('test_UnlabeledImages')
if not os.path.exists('test_ImageLatentCodes'):
    os.makedirs('test_ImageLatentCodes')

Set `N0` to the index of the first image to generate and `N` to the number of images to generate. Setting `N0` here is useful if the Google Colab instance shuts down during generation.

The cell below can be run several times to generate successive batches of images.

In [8]:
N = 50000

for k in tqdm(range(N)):
  with torch.inference_mode():
    z = torch.randn([1, G.z_dim]).to(device)
    torch.save(z, f'test_ImageLatentCodes/tensor{k}.pt')
    image = (0.5*(G(z, None)[0]+1)).cpu().detach().numpy().clip(0,1)
    plt.imsave(f'test_UnlabeledImages/img{k}.jpg', image.transpose(1, 2, 0))



  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 


  0%|                                                                            | 1/50000 [00:07<102:28:03,  7.38s/it]

Done.


100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [50:13<00:00, 16.59it/s]


NameError: name 'N0' is not defined

Archiving could have been done here, but we had this idea after all the required datasets were generated. The code that we ran simply put all the files directly into Google Drive.